In [28]:
import pandas as pd
import time
from FlightRadar24.errors import AirportNotFoundError  # Import the AirportNotFoundError
import copy
import json
from datetime import datetime


In [2]:
from FlightRadar24 import FlightRadar24API
fr_api = FlightRadar24API()

In [3]:
flights = fr_api.get_flights()
airports = fr_api.get_airports()

In [4]:
df_airports = pd.DataFrame(airports)
df_airports.to_csv("api_data/airports")

In [5]:
df_flights = pd.DataFrame(flights)
df_flights.to_csv("api_data/flights")

# Airports details

Récupère dans une liste les détails pour chaque aéroports qui se trouvent dans la liste **airports**

In [6]:
airports = pd.read_csv("api_data/airports",sep = ",")

In [151]:
airports_details3 = []
notfound = []

In [ ]:
for c in airports_bis:  
    try:
        airports_details3.append(fr_api.get_airport_details(c.split()[0][2:-1]))
        time.sleep(5)  # Wait for 5 second between API calls
    except AirportNotFoundError as e:
        notfound.append(c)

In [ ]:
airports_d = [airports_details3[i]["airport"]["pluginData"]["details"]["code"]["icao"] for i in range(len(airports_details3))]

In [ ]:
airports_bis = [icao for icao in airports['0'] if icao.split()[0][2:-1] not in airports_d]

In [ ]:
output_file = open('airports_details_final', 'w', encoding='utf-8')
for dic in airports_details3:
    json.dump(dic, output_file) 
    output_file.write("\n")

## Création du dataset final

In [7]:
airport_list = []
with open(r'airports_details_final', 'r', encoding='utf-8') as input_file:
    for line_number, line in enumerate(input_file, start=1):
        try:
            airport_list.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Erreur de décodage JSON à la ligne {line_number}: {e}")

Erreur de décodage JSON à la ligne 5454: Expecting ',' delimiter: line 1 column 1462 (char 1461)


***
En exécutant la fonction **fr_api.get_airport_details(code_airport)**, il n'a rien trouvé (code introuvable) pour certain aéroports qui se trouvent dans la liste obtenu avec **fr_api.get_airports()**

In [8]:
len(airport_list)

5453

In [9]:
len(airports)

5498

### Se familiariser avec le fichier json

In [10]:
for i in range(len(airport_list)):
    airport_list[i]['airport']['pluginData']['details'].pop('airportImages')
    airport_list[i].pop('aircraftImages')

In [11]:
print(airport_list[0]['airport']['pluginData'].keys())
print(airport_list[0]['airport']['pluginData']['details'].keys())
print(airport_list[0]['airport']['pluginData']['flightdiary'].keys())
print(airport_list[0]['airport']['pluginData']['schedule'].keys())
print(airport_list[0]['airport']['pluginData']['weather'].keys())
print(airport_list[0]['airport']['pluginData']['aircraftCount'].keys())
# print(airports_details[0]['airport']['pluginData']['runways'].keys())

dict_keys(['details', 'flightdiary', 'schedule', 'weather', 'aircraftCount', 'runways'])
dict_keys(['name', 'code', 'delayIndex', 'stats', 'position', 'timezone', 'url', 'visible'])
dict_keys(['url', 'ratings', 'comment', 'reviews', 'evaluation'])
dict_keys(['arrivals', 'departures', 'ground'])
dict_keys(['metar', 'time', 'qnh', 'dewpoint', 'humidity', 'pressure', 'sky', 'flight', 'wind', 'temp', 'elevation', 'cached'])
dict_keys(['ground', 'onGround'])


In [19]:
#Schedule qui m'intéresse: --> arrivals/departures
print(airport_list[0]['airport']['pluginData']['schedule']['departures'].keys())
for i in range(len(airport_list[0]['airport']['pluginData']['schedule']['departures']['data'])):
    for key in airport_list[0]['airport']['pluginData']['schedule']['departures']['data'][3]['flight'].keys():
        print(key)
        print(airport_list[0]['airport']['pluginData']['schedule']['departures']['data'][3]['flight'][key])
    break

# for arrival in airport_list[0]['airport']['pluginData']['schedule']['arrivals']['data']:
#     print(arrival['flight'])

dict_keys(['item', 'page', 'timestamp', 'data'])
identification
{'id': None, 'row': 5481563816, 'number': {'default': 'IB511', 'alternative': None}, 'callsign': None, 'codeshare': ['AA8546', 'AV6197', 'AY5606', 'BA7111', 'CX1862', 'LA7030', 'VY5034', 'JL9414']}
status
{'live': False, 'text': 'Estimated dep 06:30', 'icon': 'green', 'estimated': None, 'ambiguous': False, 'generic': {'status': {'text': 'estimated', 'type': 'departure', 'color': 'green', 'diverted': None}, 'eventTime': {'utc': 1710480600, 'local': 1710484200}}}
aircraft
{'model': {'code': '319', 'text': ''}, 'hex': '', 'registration': '', 'serialNo': '', 'images': None}
owner
None
airline
{'name': 'Iberia', 'code': {'iata': 'IB', 'icao': 'IBE'}, 'short': 'Iberia'}
airport
{'origin': {'timezone': {'name': 'Europe/Madrid', 'offset': 3600, 'abbr': 'CET', 'abbrName': 'Central European Time', 'isDst': False}, 'info': {'terminal': 'T1', 'baggage': None, 'gate': '3'}}, 'destination': {'code': {'iata': 'MAD', 'icao': 'LEMD'}, 'tim

### Récupération des informations pertinentes

In [22]:
country_list = []
all_airports = []
arr_flight_identification = []
arr_flight_status = []
arr_flight_aircraft_code = []
arr_flight_aircraft_name = []
arr_flight_aircraft_registration = []
arr_flight_aircraft_owner_country = []
arr_flight_aircraft_owner_airline_name = []
arr_flight_aircraft_owner_airline_code = []
dep_flight_identification = []
dep_flight_status = []
dep_flight_aircraft_code = []
dep_flight_aircraft_name = []
dep_flight_aircraft_registration = []
dep_flight_aircraft_owner_country = []
dep_flight_aircraft_owner_airline_name = []
dep_flight_aircraft_owner_airline_code = []
arr_came_from_country = []
arr_came_from_airport_name = []
arr_came_from_airport_code = []
arr_scheduled_departure = []
arr_scheduled_arrival = []
arr_real_departure = []
arr_real_arrival = []
arr_estimated_departure = []
arr_estimated_arrival = []
arr_duree = []
dep_country_destination = []
dep_airport_name_destination = []
dep_airport_code_icao_destination = []
dep_scheduled_departure = []
dep_scheduled_arrival = []
dep_real_departure = []
dep_real_arrival = []
dep_estimated_departure = []
dep_estimated_arrival = []
dep_duree = []

for i in range(len(airport_list)):
    for arrival,departure in zip(airport_list[i]['airport']['pluginData']['schedule']['arrivals']['data'],airport_list[i]['airport']['pluginData']['schedule']['departures']['data']):
        country_list.append(airport_list[i]['airport']['pluginData']['details']['position']['country']['name'])
        all_airports.append(airport_list[i]['airport']['pluginData']['details']['name'])
        try:
            arr_flight_identification.append(arrival['flight']['identification']['number']['default'])
        except (KeyError, TypeError):
            arr_flight_identification.append(None)
        try:
            arr_flight_status.append(arrival['flight']['status']['text'])
        except (KeyError, TypeError):
            arr_flight_status.append(None)
        try:
            arr_flight_aircraft_code.append(arrival['flight']['aircraft']['model']['code'])
        except (KeyError, TypeError):
            arr_flight_aircraft_code.append(None)
        try:
            arr_flight_aircraft_name.append(arrival['flight']['aircraft']['model']['text'])
        except (KeyError, TypeError):
            arr_flight_aircraft_registration.append(None)
        try:
            arr_flight_aircraft_registration.append(arrival['flight']['aircraft']['registration'])
        except (KeyError, TypeError):
            arr_flight_aircraft_registration.append(None)
        try:
            arr_flight_aircraft_owner_country.append(arrival['flight']['aircraft']['country']['name'])
        except (KeyError, TypeError):
            arr_flight_aircraft_owner_country.append(None)
        try:
            arr_flight_aircraft_owner_airline_name.append(arrival['flight']['owner']['name'])
        except (KeyError, TypeError):
            arr_flight_aircraft_owner_airline_name.append(None)
        try:
            arr_flight_aircraft_owner_airline_code.append(arrival['flight']['owner']['code']['icao'])
        except (KeyError, TypeError):
            arr_flight_aircraft_owner_airline_code.append(None)


        try:
            dep_flight_identification.append(departure['flight']['identification']['number']['default'])
        except (KeyError, TypeError):
            dep_flight_identification.append(None)
        try:
            dep_flight_status.append(departure['flight']['status']['text'])
        except (KeyError, TypeError):
            arr_flight_status.append(None)
        try:
            dep_flight_aircraft_code.append(departure['flight']['aircraft']['model']['code'])
        except (KeyError, TypeError):
            dep_flight_aircraft_code.append(None)
        try:
            dep_flight_aircraft_name.append(departure['flight']['aircraft']['model']['text'])
        except (KeyError, TypeError):
            dep_flight_aircraft_registration.append(None)
        try:
            dep_flight_aircraft_registration.append(departure['flight']['aircraft']['registration'])
        except (KeyError, TypeError):
            dep_flight_aircraft_registration.append(None)
        try:
            dep_flight_aircraft_owner_country.append(departure['flight']['aircraft']['country']['name'])
        except (KeyError, TypeError):
            dep_flight_aircraft_owner_country.append(None)
        try:
            dep_flight_aircraft_owner_airline_name.append(departure['flight']['owner']['name'])
        except (KeyError, TypeError):
            dep_flight_aircraft_owner_airline_name.append(None)
        try:
            dep_flight_aircraft_owner_airline_code.append(departure['flight']['owner']['code']['icao'])
        except (KeyError, TypeError):
            dep_flight_aircraft_owner_airline_code.append(None)


        #flight arrivals/departures details
        try:
            arr_came_from_country.append(arrival['flight']['airport']['origin']['position']['country']['name'])
        except (KeyError, TypeError):
            arr_came_from_country.append(None)
        try:
            arr_came_from_airport_name.append(arrival['flight']['airport']['origin']['name'])
        except (KeyError, TypeError):
            arr_came_from_airport_name.append(None)
        try:
            arr_came_from_airport_code.append(arrival['flight']['airport']['origin']['code']['icao'])
        except (KeyError, TypeError):
            arr_came_from_airport_code.append(None)
        try:
            arr_scheduled_departure.append(arrival['flight']['time']['scheduled']['departure'])
        except (KeyError, TypeError):
            arr_scheduled_departure.append(None)
        try:
            arr_scheduled_arrival.append(arrival['flight']['time']['scheduled']['arrival'])
        except (KeyError, TypeError):
            arr_scheduled_arrival.append(None)
        try:
            arr_real_departure.append(arrival['flight']['time']['real']['departure'])
        except (KeyError, TypeError):
            arr_real_departure.append(None)
        try:
            arr_real_arrival.append(arrival['flight']['time']['real']['arrival'])
        except (KeyError, TypeError):
            arr_real_arrival.append(None)
        try:
            arr_estimated_departure.append(arrival['flight']['time']['estimated']['departure'])
        except (KeyError, TypeError):
            arr_estimated_departure.append(None)
        try:
            arr_estimated_arrival.append(arrival['flight']['time']['estimated']['arrival'])
        except (KeyError, TypeError):
            arr_estimated_arrival.append(None)
        try:
            arr_duree.append(arrival['flight']['time']['other']['duration'])
        except (KeyError, TypeError):
            arr_duree.append(None)

        try:
            dep_country_destination.append(departure['flight']['airport']['destination']['position']['country']['name'])
        except (KeyError, TypeError):
            dep_country_destination.append(None)
        try:
            dep_airport_name_destination.append(departure['flight']['airport']['destination']['name'])
        except (KeyError, TypeError):
            dep_airport_name_destination.append(None)
        try:
            dep_airport_code_icao_destination.append(departure['flight']['airport']['destination']['code']['icao'])
        except (KeyError, TypeError):
            dep_airport_code_icao_destination.append(None)
        try:
            dep_scheduled_departure.append(departure['flight']['time']['scheduled']['departure'])
        except (KeyError, TypeError):
            dep_scheduled_departure.append(None)
        try:
            dep_scheduled_arrival.append(departure['flight']['time']['scheduled']['arrival'])
        except (KeyError, TypeError):
            dep_scheduled_arrival.append(None)
        try:
            dep_real_departure.append(departure['flight']['time']['real']['departure'])
        except (KeyError, TypeError):
            dep_real_departure.append(None)
        try:
            dep_real_arrival.append(departure['flight']['time']['real']['arrival'])
        except (KeyError, TypeError):
            dep_real_arrival.append(None)
        try:
            dep_estimated_departure.append(departure['flight']['time']['estimated']['departure'])
        except (KeyError, TypeError):
            dep_estimated_departure.append(None)
        try:
            dep_estimated_arrival.append(departure['flight']['time']['estimated']['arrival'])
        except (KeyError, TypeError):
            dep_estimated_arrival.append(None)
        try:
            dep_duree.append(departure['flight']['time']['other']['duration'])
        except (KeyError, TypeError):
            dep_duree.append(None)

In [33]:
data_arrivals = []
data_departures = []
for c, a, arr_id, arr_stat, arr_plane_code, arr_plane_name,arr_plane_reg, arr_plane_owner_c, arr_plane_owner_airline_name, arr_plane_owner_airline_code, dep_id, dep_stat, dep_plane_code, dep_plane_name,dep_plane_reg, dep_plane_owner_c, dep_plane_owner_airline_name, dep_plane_owner_airline_code, arr_from, arr_name, arr_code, arr_sd, arr_sa, arr_rd, arr_ra, arr_ed, arr_ea, arr_d,dep_dest, dep_name, dep_code, dep_sd,dep_sa, dep_rd, dep_ra,dep_ed, dep_ea, dep_d in zip(country_list,all_airports,arr_flight_identification,arr_flight_status,arr_flight_aircraft_code,arr_flight_aircraft_name,arr_flight_aircraft_registration,arr_flight_aircraft_owner_country,arr_flight_aircraft_owner_airline_name,arr_flight_aircraft_owner_airline_code,dep_flight_identification,dep_flight_status,dep_flight_aircraft_code,dep_flight_aircraft_name,dep_flight_aircraft_registration,dep_flight_aircraft_owner_country,dep_flight_aircraft_owner_airline_name,dep_flight_aircraft_owner_airline_code,arr_came_from_country,arr_came_from_airport_name,arr_came_from_airport_code,arr_scheduled_departure,arr_scheduled_arrival,arr_real_departure,arr_real_arrival,arr_estimated_departure,arr_estimated_arrival,arr_duree,dep_country_destination,dep_airport_name_destination,dep_airport_code_icao_destination,dep_scheduled_departure,dep_scheduled_arrival,dep_real_departure,dep_real_arrival,dep_estimated_departure,dep_estimated_arrival,dep_duree):
    if arr_sd is not None:
        arr_sd_formatted = datetime.utcfromtimestamp(arr_sd).strftime('%Y-%m-%d %H:%M:%S')
    else:
        arr_sd_formatted = None
    if arr_sa is not None:
        arr_sa_formatted = datetime.utcfromtimestamp(arr_sa).strftime('%Y-%m-%d %H:%M:%S')
    else:
        arr_sa_formatted = None
    if arr_rd is not None:
        arr_rd_formatted = datetime.utcfromtimestamp(arr_rd).strftime('%Y-%m-%d %H:%M:%S')
    else:
        arr_rd_formatted = None
    if arr_ra is not None:
        arr_ra_formatted = datetime.utcfromtimestamp(arr_ra).strftime('%Y-%m-%d %H:%M:%S')
    else:
        arr_ra_formatted = None
    if arr_ed is not None:
        arr_ed_formatted = datetime.utcfromtimestamp(arr_ed).strftime('%Y-%m-%d %H:%M:%S')
    else:
        arr_ed_formatted = None
    if arr_ea is not None:
        arr_ea_formatted = datetime.utcfromtimestamp(arr_ea).strftime('%Y-%m-%d %H:%M:%S')
    else:
        arr_ea_formatted = None
    if arr_d is not None:
        arr_d_formatted = datetime.utcfromtimestamp(arr_d).strftime('%Y-%m-%d %H:%M:%S')
    else:
        arr_d_formatted = None
    if dep_sd is not None:
        dep_sd_formatted = datetime.utcfromtimestamp(dep_sd).strftime('%Y-%m-%d %H:%M:%S')
    else:
        dep_sd_formatted = None
    if dep_sa is not None:
        dep_sa_formatted = datetime.utcfromtimestamp(dep_sa).strftime('%Y-%m-%d %H:%M:%S')
    else:
        dep_sa_formatted = None
    if dep_rd is not None:
        dep_rd_formatted = datetime.utcfromtimestamp(dep_rd).strftime('%Y-%m-%d %H:%M:%S')
    else:
        dep_rd_formatted = None
    if dep_ra is not None:
        dep_ra_formatted = datetime.utcfromtimestamp(dep_ra).strftime('%Y-%m-%d %H:%M:%S')
    else:
        dep_ra_formatted = None
    if dep_ed is not None:
        dep_ed_formatted = datetime.utcfromtimestamp(dep_ed).strftime('%Y-%m-%d %H:%M:%S')
    else:
        dep_ed_formatted = None
    if dep_ea is not None:
        dep_ea_formatted = datetime.utcfromtimestamp(dep_ea).strftime('%Y-%m-%d %H:%M:%S') 
    else:
        dep_ea_formatted = None
    if dep_d is not None:
        dep_d_formatted = datetime.utcfromtimestamp(dep_d).strftime('%Y-%m-%d %H:%M:%S')
    else:
        dep_d_formatted = None
    data_arrivals.append([c, a, arr_id, arr_stat, arr_plane_code, arr_plane_name,arr_plane_reg, arr_plane_owner_c, arr_plane_owner_airline_name, arr_plane_owner_airline_code, arr_from, arr_name, arr_code, arr_sd_formatted, arr_sa_formatted, arr_rd_formatted, arr_ra_formatted, arr_ed_formatted,arr_ea_formatted, arr_d_formatted])
    data_departures.append([c,a,dep_id, dep_stat, dep_plane_code, dep_plane_name,dep_plane_reg, dep_plane_owner_c, dep_plane_owner_airline_name, dep_plane_owner_airline_code,dep_dest, dep_name, dep_code, dep_sd_formatted,dep_sa_formatted, dep_rd_formatted, dep_ra_formatted,dep_ed_formatted,dep_ea_formatted, dep_d_formatted])

In [35]:
df_arrivals = pd.DataFrame(data_arrivals, columns=["Country","Airport", "Flight identification", "Flight Status", "Aircraft Code", "Aircraft Name", "Aircraft Registration","Aircraft owner (Country)","Airline Name","Airline Code", "Departure Country","Departure Airport Name","Departure Airport Code","Scheduled Departure","Scheduled Arrival","Real Departure","Real Arrival","Estimated Departure","Estimated Arrival","Flight Duration"])
df_departures = pd.DataFrame(data_departures, columns=["Country","Airport", "Flight identification", "Flight Status", "Aircraft Code", "Aircraft Name", "Aircraft Registration","Aircraft owner (Country)","Airline Name","Airline Code", "Destination Country","Destination Airport Name","Destination Airport Code","Scheduled Departure","Scheduled Arrival","Real Departure","Real Arrival","Estimated Departure","Estimated Arrival","Flight Duration"])

In [47]:
df_arrivals_sorted = df_arrivals.sort_values(by=['Country', 'Airport']).reset_index(drop=True)
df_departures_sorted = df_departures.sort_values(by=['Country', 'Airport']).reset_index(drop=True)

In [48]:
df_arrivals_sorted

,Country,Airport,Flight identification,Flight Status,Aircraft Code,Aircraft Name,Aircraft Registration,Aircraft owner (Country),Airline Name,Airline Code,Departure Country,Departure Airport Name,Departure Airport Code,Scheduled Departure,Scheduled Arrival,Real Departure,Real Arrival,Estimated Departure,Estimated Arrival,Flight Duration
0,Afghanistan,Herat International Airport,RQ101,Scheduled,737,,,None,None,None,Afghanistan,Kabul International Airport,OAKB,2024-03-15 05:00:00,2024-03-15 06:30:00,None,None,None,None,None
1,Afghanistan,Herat International Airport,RQ147,Scheduled,737,,,None,None,None,Afghanistan,Mazar-I-Sharif International Airport,OAMS,2024-03-15 07:15:00,2024-03-15 08:30:00,None,None,None,None,None
2,Afghanistan,Herat International Airport,FG251,Scheduled,737,,,None,None,None,Afghanistan,Kabul International Airport,OAKB,2024-03-15 07:30:00,2024-03-15 08:40:00,None,None,None,None,None
3,Afghanistan,Herat International Airport,RQ101,Scheduled,737,,SE-MFY,None,None,None,Afghanistan,Kabul International Airport,OAKB,2024-03-16 05:00:00,2024-03-16 06:30:00,None,None,None,None,None
4,Afghanistan,Herat International Airport,FG251,Scheduled,737,Boeing 737 MAX 8,,None,None,None,Afghanistan,Kabul International Airport,OAKB,2024-03-16 07:30:00,2024-03-16 08:40:00,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77572,Zimbabwe,Victoria Falls Airport,4Z494,Scheduled,E90,Mitsubishi CRJ-900LR,HB-IJO,None,None,None,South Africa,Johannesburg OR Tambo International Airport,FAOR,2024-03-16 09:35:00,2024-03-16 11:20:00,None,None,None,None,None
77573,Zimbabwe,Victoria Falls Airport,4Z476,Scheduled,ER3,Boeing 787-8 Dreamliner,,None,None,None,South Africa,Nelspruit Kruger Mpumalanga International Airport,FAKN,2024-03-16 09:35:00,2024-03-16 11:25:00,None,None,None,None,None
77574,Zimbabwe,Victoria Falls Airport,SA40,Scheduled,320,Boeing 767-3Z9(ER),OE-LWG,None,None,None,South Africa,Johannesburg OR Tambo International Airport,FAOR,2024-03-16 09:50:00,2024-03-16 11:25:00,None,None,None,None,None
77575,Zimbabwe,Victoria Falls Airport,5Z764,Scheduled,CR1,Embraer E175STD,,None,None,None,South Africa,Johannesburg OR Tambo International Airport,FAOR,2024-03-16 10:40:00,2024-03-16 12:20:00,None,None,None,None,None


In [49]:
df_departures_sorted

,Country,Airport,Flight identification,Flight Status,Aircraft Code,Aircraft Name,Aircraft Registration,Aircraft owner (Country),Airline Name,Airline Code,Destination Country,Destination Airport Name,Destination Airport Code,Scheduled Departure,Scheduled Arrival,Real Departure,Real Arrival,Estimated Departure,Estimated Arrival,Flight Duration
0,Afghanistan,Herat International Airport,RQ102,Scheduled,737,,,None,None,None,Afghanistan,Kabul International Airport,OAKB,2024-03-15 07:15:00,2024-03-15 08:30:00,None,None,None,None,None
1,Afghanistan,Herat International Airport,RQ148,Scheduled,737,,,None,None,None,Afghanistan,Mazar-I-Sharif International Airport,OAMS,2024-03-15 09:15:00,2024-03-15 10:30:00,None,None,None,None,None
2,Afghanistan,Herat International Airport,FG252,Scheduled,737,,,None,None,None,Afghanistan,Kabul International Airport,OAKB,2024-03-15 09:30:00,2024-03-15 10:45:00,None,None,None,None,None
3,Afghanistan,Herat International Airport,RQ102,Scheduled,737,,,None,None,None,Afghanistan,Kabul International Airport,OAKB,2024-03-16 07:15:00,2024-03-16 08:30:00,None,None,None,None,None
4,Afghanistan,Herat International Airport,FG252,Scheduled,737,,,None,None,None,Afghanistan,Kabul International Airport,OAKB,2024-03-16 09:30:00,2024-03-16 10:45:00,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77572,Zimbabwe,Victoria Falls Airport,SA41,Scheduled,320,Airbus A320-232,,None,None,None,South Africa,Johannesburg OR Tambo International Airport,FAOR,2024-03-16 12:10:00,2024-03-16 13:50:00,None,None,None,None,None
77573,Zimbabwe,Victoria Falls Airport,FN8501,Scheduled,ER4,Airbus A220-300,HB-JBC,None,None,None,South Africa,Johannesburg OR Tambo International Airport,FAOR,2024-03-16 12:15:00,2024-03-16 14:00:00,None,None,None,None,None
77574,Zimbabwe,Victoria Falls Airport,5Z765,Scheduled,CR1,Embraer E195LR,9H-QBF,None,None,None,South Africa,Johannesburg OR Tambo International Airport,FAOR,2024-03-16 13:05:00,2024-03-16 14:40:00,None,None,None,None,None
77575,Zimbabwe,Victoria Falls Airport,FN8004,Scheduled,ER4,Airbus A320-214,D-AILE,None,None,None,Zimbabwe,Harare Robert Gabriel Mugabe International Air...,FVRG,2024-03-16 13:15:00,2024-03-16 14:20:00,None,None,None,None,None


In [50]:
df_arrivals_sorted.to_csv(f'./data/df_arrivals.csv',index=False)
df_departures_sorted.to_csv(f'./data/df_departures.csv',index=False)